In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jan  1 16:24:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!gdown --id 1YuzCW6s56tV409JRXaAMKFdIkK9xIACw --output "data.zip"
!unzip -q "data.zip"

Downloading...
From: https://drive.google.com/uc?id=1YuzCW6s56tV409JRXaAMKFdIkK9xIACw
To: /content/data.zip
100% 1.13G/1.13G [00:05<00:00, 217MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
import argparse

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import Sampler

import csv
import random
import numpy as np
import pandas as pd

from PIL import Image
filenameToPILImage = lambda x: Image.open(x)

# mini-Imagenet dataset
class MiniDataset(Dataset):
    def __init__(self, csv_path, data_dir):
        self.data_dir = data_dir
        self.data_df = pd.read_csv(csv_path).set_index("id")

        self.transform = transforms.Compose([
            filenameToPILImage,
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

    def __getitem__(self, index):
        path = self.data_df.loc[index, "filename"]
        label = self.data_df.loc[index, "label"]
        image = self.transform(os.path.join(self.data_dir, path))
        return image, label

    def __len__(self):
        return len(self.data_df)

In [ ]:
from torch import nn
import torch


class Convnet(nn.Module):
    def __init__(self, in_channels=3, hid_channels=64, out_channels=64):
        super().__init__()
        self.encoder = nn.Sequential(
            conv_block(in_channels, hid_channels),
            conv_block(hid_channels, hid_channels),
            conv_block(hid_channels, hid_channels),
            conv_block(hid_channels, out_channels)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)

def conv_block(in_channels, out_channels):
    bn = nn.BatchNorm2d(out_channels)
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        bn,
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

class Protonet(nn.Module):
    def __init__(self, in_channels=3, hid_channels=64, out_channels=64):
        super().__init__()
        self.conv = Convnet(in_channels, hid_channels, out_channels)
        self.mlp = nn.Sequential(
            nn.Linear(1600, 800),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(800, 800),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(800, 400)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.mlp(x)
        return x

In [ ]:
from torch.utils.data import Sampler
import numpy as np
import pandas as pd


class NShotTaskSampler(Sampler):
    def __init__(self, csv_path, episodes_per_epoch, N_way, N_shot, N_query):
        self.data_df = pd.read_csv(csv_path)
        self.N_way = N_way
        self.N_shot = N_shot
        self.N_query = N_query
        self.episodes_per_epoch = episodes_per_epoch

    def __iter__(self):
        for _ in range(self.episodes_per_epoch):
            batch = []
            episode_classes = np.random.choice(self.data_df['label'].unique(), size=self.N_way, replace=False)

            support = []
            query = []

            for k in episode_classes:
                ind = self.data_df[self.data_df['label'] == k]['id'].sample(self.N_shot + self.N_query).values
                support = support + list(ind[:self.N_shot])
                query = query + list(ind[self.N_shot:])

            batch = support + query

            yield np.stack(batch)

    def __len__(self):
        return self.episodes_per_epoch


class GeneratorSampler(Sampler):
    def __init__(self, episode_file_path):
        episode_df = pd.read_csv(episode_file_path).set_index("episode_id")
        self.sampled_sequence = episode_df.values.flatten().tolist()

    def __iter__(self):
        return iter(self.sampled_sequence) 

    def __len__(self):
        return len(self.sampled_sequence)

In [ ]:
import torch
import torch.nn as nn

def pairwise_distances(x, y, matching_fn='l2', parametric=None):
    n_x = x.shape[0]
    n_y = y.shape[0]

    if matching_fn == 'l2':
        distances = (
                x.unsqueeze(1).expand(n_x, n_y, -1) -
                y.unsqueeze(0).expand(n_x, n_y, -1)
        ).pow(2).sum(dim=2)
        return distances
    elif matching_fn == 'cosine':
        cos = nn.CosineSimilarity(dim=2, eps=1e-6)
        cosine_similarities = cos(x.unsqueeze(1).expand(n_x, n_y, -1), y.unsqueeze(0).expand(n_x, n_y, -1))

        return 1 - cosine_similarities
    elif matching_fn == 'parametric':
        x_exp = x.unsqueeze(1).expand(n_x, n_y, -1).reshape(n_x*n_y, -1)
        y_exp = y.unsqueeze(0).expand(n_x, n_y, -1).reshape(n_x*n_y, -1)
        
        distances = parametric(torch.cat([x_exp, y_exp], dim=-1))
        
        return distances.reshape(n_x, n_y)

    else:
        raise(ValueError('Unsupported similarity function'))

In [ ]:
import os
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as vutils
from torch.optim.lr_scheduler import StepLR
# from model import Protonet
# from utils import pairwise_distances


class Solver(object):
    def __init__(self, config, train_loader, val_loader):
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device('cuda' if self.use_cuda else 'cpu')
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.N_way_train = config.N_way_train
        self.N_shot_train = config.N_shot_train
        self.N_query_train = config.N_query_train
        self.N_way_val = config.N_way_val
        self.N_shot_val = config.N_shot_val
        self.N_query_val = config.N_query_val
        self.matching_fn = config.matching_fn

        self.num_epochs = config.num_epochs
        self.resume_iter = config.resume_iter
        self.lr = config.lr
        self.num_steps_decay = config.num_steps_decay
        self.beta1 = config.beta1
        self.beta2 = config.beta2
        self.weight_decay = config.weight_decay
        self.exp_name = config.name
        os.makedirs(config.ckp_dir, exist_ok=True)
        self.ckp_dir = os.path.join(config.ckp_dir, self.exp_name)
        os.makedirs(self.ckp_dir, exist_ok=True)
        self.log_interval = config.log_interval
        self.ckp_interval = config.ckp_interval

        self.use_wandb = config.use_wandb
        
        self.build_model()

    def build_model(self):
        self.model = Protonet().to(self.device)
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr,  betas=[self.beta1, self.beta2], weight_decay=self.weight_decay)
        if self.matching_fn == 'parametric':
            self.parametric = nn.Sequential(
                nn.Linear(800, 400),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(400, 1)
            ).to(self.device)
            self.optimizer = torch.optim.AdamW(list(self.model.parameters()) + list(self.parametric.parameters()), lr=self.lr,  betas=[self.beta1, self.beta2], weight_decay=self.weight_decay)
        self.scheduler = StepLR(self.optimizer, step_size=self.num_steps_decay, gamma=0.9)

    def save_checkpoint(self, step):
        state = {'state_dict': self.model.state_dict(),
                 'optimizer' : self.optimizer.state_dict()}
        if self.matching_fn == 'parametric':
            state = {'state_dict': self.model.state_dict(),
                     'optimizer' : self.optimizer.state_dict(),
                     'parametric': self.parametric.state_dict()}

        new_checkpoint_path = os.path.join(self.ckp_dir, '{}-protonet.pth'.format(step + 1))
        torch.save(state, new_checkpoint_path)
        print('model saved to %s' % new_checkpoint_path)

    def load_checkpoint(self, resume_iter):
        print('Loading the trained models from step {}...'.format(resume_iter))
        new_checkpoint_path = os.path.join(self.ckp_dir, '{}-protonet.pth'.format(resume_iter))
        state = torch.load(new_checkpoint_path)
        self.model.load_state_dict(state['state_dict'])
        self.optimizer.load_state_dict(state['optimizer'])
        if self.matching_fn == 'parametric':
            self.parametric.load_state_dict(state['parametric'])
        print('model loaded from %s' % new_checkpoint_path)
    
    def train(self):
        criterion = nn.CrossEntropyLoss()

        best_mean = 0
        iteration = 0

        if self.resume_iter:
            print("resuming step %d ..."% self.resume_iter)
            iteration = self.resume_iter
            self.load_checkpoint(self.resume_iter)
            loss, mean, std = self.eval()
            if mean > best_mean:
                best_mean = mean

        episodic_acc = []

        for ep in range(self.num_epochs):
            self.model.train()

            for batch_idx, (data, target) in enumerate(self.train_loader):
                data = data.to(self.device)
                self.optimizer.zero_grad()
                
                support_input = data[:self.N_way_train * self.N_shot_train,:,:,:] 
                query_input   = data[self.N_way_train * self.N_shot_train:,:,:,:]

                label_encoder = {target[i * self.N_shot_train] : i for i in range(self.N_way_train)}
                query_label = torch.cuda.LongTensor([label_encoder[class_name] for class_name in target[self.N_way_train * self.N_shot_train:]])

                support = self.model(support_input)
                queries = self.model(query_input)
                prototypes = support.reshape(self.N_way_train, self.N_shot_train, -1).mean(dim=1)

                if self.matching_fn == 'parametric':
                    distances = pairwise_distances(queries, prototypes, self.matching_fn, self.parametric)

                else:
                    distances = pairwise_distances(queries, prototypes, self.matching_fn)

                loss = criterion(-distances, query_label)
                loss.backward()
                self.optimizer.step()

                y_pred = (-distances).softmax(dim=1).max(1, keepdim=True)[1]
                episodic_acc.append(1. * y_pred.eq(query_label.view_as(y_pred)).sum().item() / len(query_label))


                if (iteration + 1) % self.log_interval == 0:
                    episodic_acc = np.array(episodic_acc)
                    mean = episodic_acc.mean()
                    std = episodic_acc.std()

                    print('Epoch: {:3d} [{:d}/{:d}]\tIteration: {:5d}\tLoss: {:.6f}\tAccuracy: {:.2f} +- {:.2f} %'.format(
                        ep, (batch_idx + 1), len(self.train_loader), iteration + 1, loss.item(), 
                        mean * 100, 1.96 * std / (self.log_interval)**(1/2) * 100))

                    if self.use_wandb:
                        import wandb
                        wandb.log({"loss": loss.item(),
                                   "acc_mean": mean * 100,
                                   "acc_ci": 1.96 * std / (self.log_interval)**(1/2) * 100,
                                   'lr': self.optimizer.param_groups[0]['lr']
                                   }, 
                                   step=iteration+1)

                    episodic_acc = []


                if (iteration + 1) % self.ckp_interval == 0:
                    loss, mean, std = self.eval()
                    if mean > best_mean:
                        best_mean = mean
                        self.save_checkpoint(iteration)
                        if self.use_wandb:
                            wandb.run.summary["best_accuracy"] = best_mean * 100

                    if self.use_wandb:
                        import wandb
                        wandb.log({"val_loss": loss,
                                   "val_acc_mean": mean * 100,
                                   "val_acc_ci": 1.96 * std / (600)**(1/2) * 100}, 
                                   step=iteration+1, commit=False)

                iteration += 1

            self.scheduler.step()




    def eval(self):
        criterion = nn.CrossEntropyLoss()
        self.model.eval()
        episodic_acc = []
        loss = []
        
        with torch.no_grad():
            for b_idx, (data, target) in enumerate(self.val_loader):
                data = data.to(self.device)
                support_input = data[:self.N_way_val * self.N_shot_val,:,:,:] 
                query_input = data[self.N_way_val * self.N_shot_val:,:,:,:]

                label_encoder = {target[i * self.N_shot_val] : i for i in range(self.N_way_val)}
                query_label = torch.cuda.LongTensor([label_encoder[class_name] for class_name in target[self.N_way_val * self.N_shot_val:]])

                support = self.model(support_input)
                queries = self.model(query_input)
                prototypes = support.reshape(self.N_way_val, self.N_shot_val, -1).mean(dim=1)

                if self.matching_fn == 'parametric':
                    distances = pairwise_distances(queries, prototypes, self.matching_fn, self.parametric)
                else:
                    distances = pairwise_distances(queries, prototypes, self.matching_fn)
                    
                loss.append(criterion(-distances, query_label).item())
                y_pred = (-distances).softmax(dim=1).max(1, keepdim=True)[1]
                episodic_acc.append(1. * y_pred.eq(query_label.view_as(y_pred)).sum().item() / len(query_label))

        loss = np.array(loss)
        episodic_acc = np.array(episodic_acc)
        loss = loss.mean()
        mean = episodic_acc.mean()
        std = episodic_acc.std()

        print('\nLoss: {:.6f}\tAccuracy: {:.2f} +- {:.2f} %\n'.format(loss,mean * 100, 1.96 * std / (600)**(1/2) * 100))

        return loss, mean, std

In [ ]:
import os
import sys
import argparse

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import Sampler

import csv
import random
import numpy as np
import pandas as pd
from argparse import Namespace

from PIL import Image
filenameToPILImage = lambda x: Image.open(x)

# from dataset import MiniDataset
# from samplers import GeneratorSampler, NShotTaskSampler
# from solver import Solver

# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(SEED)
np.random.seed(SEED)

def worker_init_fn(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)

def parse_args():
    # parser = argparse.ArgumentParser(description="Few shot learning")

    # # training configuration.
    # parser.add_argument('--episodes_per_epoch', default=600, type=int, help='episodes per epoch')
    # parser.add_argument('--N_way_train', default=5, type=int, help='N_way (default: 5) for training')
    # parser.add_argument('--N_shot_train', default=1, type=int, help='N_shot (default: 1) for training')
    # parser.add_argument('--N_query_train', default=15, type=int, help='N_query (default: 15) for training')
    # parser.add_argument('--N_way_val', default=5, type=int, help='N_way (default: 5) for val')
    # parser.add_argument('--N_shot_val', default=1, type=int, help='N_shot (default: 1) for val')
    # parser.add_argument('--N_query_val', default=15, type=int, help='N_query (default: 15) for val')
    # parser.add_argument('--matching_fn', default='l2', type=str, choices=['l2', 'cosine', 'parametric'], help='distance matching function')

    # # optimizer configuration
    # parser.add_argument("--lr", help="the learning rate", default=1e-4, type=float)
    # parser.add_argument('--num_steps_decay', type=int, default=40, help='number of steps for decaying lr')
    # parser.add_argument('--beta1', type=float, default=0.9, help='beta1 for Adam optimizer')
    # parser.add_argument('--beta2', type=float, default=0.999, help='beta2 for Adam optimizer')
    # parser.add_argument('--weight_decay', type=float, default=1e-2, help='weight_decay for Adam optimizer')

    # # path.
    # parser.add_argument('--train_csv', type=str, default='../hw4_data/train.csv', help="Training images csv file")
    # parser.add_argument('--train_data_dir', type=str, default='../hw4_data/train', help="Training images directory")
    # parser.add_argument('--val_csv', type=str, default='../hw4_data/val.csv', help="val images csv file")
    # parser.add_argument('--val_data_dir', type=str, default='../hw4_data/val', help="val images directory")
    # parser.add_argument('--val_testcase_csv', type=str, default='../hw4_data/val_testcase.csv', help="val test case csv")
    # parser.add_argument('--ckp_dir', default='ckpt/', type=str, help='Checkpoint path', required=False)
    # parser.add_argument('--name', default='', type=str, help='Name for saving model')

    # # Step size.
    # parser.add_argument('--num_epochs', type=int, default=100, help='number of total epochs')
    # parser.add_argument('--resume_iter', type=int, default=0, help='resume training from this epoch')
    # parser.add_argument('--log_interval', type=int, default=300)
    # parser.add_argument('--ckp_interval', type=int, default=600)

    # # Others
    # parser.add_argument("--use_wandb", help="log training with wandb, "
    #     "requires wandb, install with \"pip install wandb\"", action="store_true")

    # return parser.parse_args()
    
    parameters = {
        'episodes_per_epoch': 100,
        'N_way_train': 5,
        'N_shot_train': 10,
        'N_query_train': 15,
        'N_way_val': 5,
        'N_shot_val': 1,
        'N_query_val': 15,
        'matching_fn': 'parametric',

        'lr': 1e-4,
        'num_steps_decay': 40,
        'beta1': 0.9,
        'beta2': 0.999,
        'weight_decay': 1e-2,

        'train_csv': '/content/hw4_data/mini/train.csv',
        'train_data_dir': '/content/hw4_data/mini/train',
        'val_csv': '/content/hw4_data/mini/val.csv',
        'val_data_dir': '/content/hw4_data/mini/val',
        'val_testcase_csv': '/content/hw4_data/mini/val_testcase.csv',
        'ckp_dir': './ckpt_cos',
        'name': '',

        'num_epochs': 200,
        'resume_iter': 0,
        'log_interval': 300,
        'ckp_interval': 600,

        'use_wandb': False,

    }
    config = Namespace(**parameters)

    return config
    




if __name__=='__main__':
    args = parse_args()

    if args.use_wandb:
        import wandb
        wandb.init(project="few-shot-learning", config=args)
        args = wandb.config
        print(args)

    train_dataset = MiniDataset(args.train_csv, args.train_data_dir)
    val_dataset = MiniDataset(args.val_csv, args.val_data_dir)

    train_loader = DataLoader(
        train_dataset,
        num_workers=3, pin_memory=False, worker_init_fn=worker_init_fn,
        batch_sampler=NShotTaskSampler(args.train_csv, args.episodes_per_epoch, args.N_way_train, args.N_shot_train, args.N_query_train))

    val_loader = DataLoader(
        val_dataset, batch_size=args.N_way_val * (args.N_query_val + args.N_shot_val),
        num_workers=3, pin_memory=False, worker_init_fn=worker_init_fn,
        sampler=GeneratorSampler(args.val_testcase_csv))

    solver = Solver(args, train_loader, val_loader)

    solver.train()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch:   2 [100/100]	Iteration:   300	Loss: 1.413023	Accuracy: 26.13 +- 0.85 %
Epoch:   5 [100/100]	Iteration:   600	Loss: 1.398427	Accuracy: 33.86 +- 0.90 %

Loss: 1.583953	Accuracy: 27.23 +- 0.56 %

model saved to ./ckpt_cos/600-protonet.pth
Epoch:   8 [100/100]	Iteration:   900	Loss: 1.522658	Accuracy: 35.12 +- 0.91 %
Epoch:  11 [100/100]	Iteration:  1200	Loss: 1.429200	Accuracy: 35.57 +- 0.91 %

Loss: 1.643131	Accuracy: 28.56 +- 0.58 %

model saved to ./ckpt_cos/1200-protonet.pth
Epoch:  14 [100/100]	Iteration:  1500	Loss: 1.520076	Accuracy: 37.32 +- 0.85 %
Epoch:  17 [100/100]	Iteration:  1800	Loss: 1.110024	Accuracy: 38.17 +- 0.85 %

Loss: 1.626902	Accuracy: 28.64 +- 0.58 %

model saved to ./ckpt_cos/1800-protonet.pth
Epoch:  20 [100/100]	Iteration:  2100	Loss: 1.287106	Accuracy: 39.37 +- 0.90 %
Epoch:  23 [100/100]	Iteration:  2400	Loss: 1.416582	Accuracy: 40.58 +- 0.96 %

Loss: 1.635383	Accuracy: 29.46 +- 0.62 %

model saved to ./ckpt_cos/2400-protonet.pth
Epoch:  26 [100/100]	

In [ ]:
! cp -r /content/drive/MyDrive/DLCV/HW4/p1_models/15600-protonet.pth /content/

In [ ]:
import os
import sys
import argparse

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import Sampler

import csv
import random
import numpy as np
import pandas as pd
from argparse import Namespace

from PIL import Image
filenameToPILImage = lambda x: Image.open(x)

# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(SEED)
np.random.seed(SEED)

class Convnet(nn.Module):
    def __init__(self, in_channels=3, hid_channels=64, out_channels=64):
        super().__init__()
        self.encoder = nn.Sequential(
            conv_block(in_channels, hid_channels),
            conv_block(hid_channels, hid_channels),
            conv_block(hid_channels, hid_channels),
            conv_block(hid_channels, out_channels)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)

def conv_block(in_channels, out_channels):
    bn = nn.BatchNorm2d(out_channels)
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        bn,
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

class Protonet(nn.Module):
    def __init__(self, in_channels=3, hid_channels=64, out_channels=64):
        super().__init__()
        self.conv = Convnet(in_channels, hid_channels, out_channels)
        self.mlp = nn.Sequential(
            nn.Linear(1600, 800),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(800, 800),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(800, 400)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.mlp(x)
        return x

def worker_init_fn(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)

# mini-Imagenet dataset
class MiniDataset(Dataset):
    def __init__(self, csv_path, data_dir):
        self.data_dir = data_dir
        self.data_df = pd.read_csv(csv_path).set_index("id")

        self.transform = transforms.Compose([
            filenameToPILImage,
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

    def __getitem__(self, index):
        path = self.data_df.loc[index, "filename"]
        label = self.data_df.loc[index, "label"]
        image = self.transform(os.path.join(self.data_dir, path))
        return image, label

    def __len__(self):
        return len(self.data_df)

class GeneratorSampler(Sampler):
    def __init__(self, episode_file_path):
        episode_df = pd.read_csv(episode_file_path).set_index("episode_id")
        self.sampled_sequence = episode_df.values.flatten().tolist()

    def __iter__(self):
        return iter(self.sampled_sequence) 

    def __len__(self):
        return len(self.sampled_sequence)

def pairwise_distances(x, y, matching_fn='parametric', parametric=None):
    n_x = x.shape[0]
    n_y = y.shape[0]
    x_exp = x.unsqueeze(1).expand(n_x, n_y, -1).reshape(n_x*n_y, -1)
    y_exp = y.unsqueeze(0).expand(n_x, n_y, -1).reshape(n_x*n_y, -1)
    
    distances = parametric(torch.cat([x_exp, y_exp], dim=-1))
    
    return distances.reshape(n_x, n_y)

def predict(args, model, data_loader):
    for _, m in model.items():
        m.eval()

    prediction_results = []
    episodic_acc = []


    with torch.no_grad():
        # each batch represent one episode (support data + query data)
        for i, (data, target) in enumerate(data_loader):
            data = data.cuda()
            # split data into support and query data
            support_input = data[:args.N_way * args.N_shot,:,:,:] 
            query_input   = data[args.N_way * args.N_shot:,:,:,:]

            # create the relative label (0 ~ N_way-1) for query data
            # label_encoder = {target[i * args.N_shot] : i for i in range(args.N_way)}
            # query_label = torch.cuda.LongTensor([label_encoder[class_name] for class_name in target[args.N_way * args.N_shot:]])

            #  extract the feature of support and query data
            support = model['proto'](support_input)
            queries = model['proto'](query_input)

            #  calculate the prototype for each class according to its support data
            prototypes = support.reshape(args.N_way, args.N_shot, -1).mean(dim=1)
            distances = pairwise_distances(queries, prototypes, args.matching_fn, model['parametric'])

            #  classify the query data depending on the its distense with each prototype
            y_pred = (-distances).softmax(dim=1).max(1, keepdim=True)[1]
            prediction_results.append(y_pred.reshape(-1))

    return prediction_results

def parse_args():
    # parser = argparse.ArgumentParser(description="Few shot learning")
    # parser.add_argument('--N-way', default=5, type=int, help='N_way (default: 5)')
    # parser.add_argument('--N-shot', default=1, type=int, help='N_shot (default: 1)')
    # parser.add_argument('--N-query', default=15, type=int, help='N_query (default: 15)')
    # parser.add_argument('--M_aug', default=10, type=int, help='M_augmentation (default: 10)')
    # parser.add_argument('--matching_fn', default='parametric', type=str, help='distance matching function')
    # parser.add_argument('--load', type=str, help="Model checkpoint path")
    # parser.add_argument('--test_csv', default='/content/hw4_data/mini/val.csv', type=str, help="Testing images csv file")
    # parser.add_argument('--test_data_dir', default='/content/hw4_data/mini/val', type=str, help="Testing images directory")
    # parser.add_argument('--testcase_csv', default='/content/hw4_data/mini/val_testcase.csv', type=str, help="Test case csv")
    # parser.add_argument('--output_csv', type=str, help="Output filename")

    # return parser.parse_args()
    parameter = {
        'N_way': 5,
        'N_shot': 1,
        'N_query': 15,
        'M_aug': 10,
        'matching_fn': 'parametric',  #cosine parametric l2
        'load': '/content/15600-protonet.pth',
        'test_csv': '/content/hw4_data/mini/val.csv',
        'test_data_dir': '/content/hw4_data/mini/val',
        'testcase_csv': '/content/hw4_data/mini/val_testcase.csv',
        'output_csv': './output2.csv',
    }
    config = Namespace(**parameter)
    return config

if __name__=='__main__':
    args = parse_args()

    test_dataset = MiniDataset(args.test_csv, args.test_data_dir)

    test_loader = DataLoader(
        test_dataset, batch_size=args.N_way * (args.N_query + args.N_shot),
        num_workers=3, pin_memory=False, worker_init_fn=worker_init_fn,
        sampler=GeneratorSampler(args.testcase_csv))

    # TODO: load your model
    state = torch.load(args.load)
    model = {}
    model['proto'] = Protonet().cuda()
    model['proto'].load_state_dict(state['state_dict'])
    model['parametric'] = nn.Sequential(
        nn.Linear(800, 400),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(400, 1)
    ).cuda()
    model['parametric'].load_state_dict(state['parametric'])

    prediction_results = predict(args, model, test_loader)

    if os.path.dirname(args.output_csv):
        os.makedirs(os.path.dirname(args.output_csv), exist_ok=True)
    # TODO: output your prediction to csv
    with open(args.output_csv, 'w') as out_file:
        line = 'episode_id'
        for i in range(args.N_way*args.N_query):
            line += ',query%d' % (i)
        line += '\n'
        out_file.write(line)

        for i, prediction in enumerate(prediction_results):
            line = '%d' % (i)
            for j in prediction:
                line += ',%d' % j
            line += '\n'
            out_file.write(line)

#Reference: https://github.com/kai860115/DLCV2020-FALL/tree/main/hw4

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
import sys, csv
import numpy as np
path1 = '/content/output2.csv'
path2 = '/content/hw4_data/mini/val_testcase_gt.csv'

# read your prediction file
with open(path1, mode='r') as pred:
    reader = csv.reader(pred)
    next(reader, None)  # skip the headers
    pred_dict = {int(rows[0]): np.array(rows[1:]).astype(int) for rows in reader}

# read ground truth data
with open(path2, mode='r') as gt:
    reader = csv.reader(gt)
    next(reader, None)  # skip the headers
    gt_dict = {int(rows[0]): np.array(rows[1:]).astype(int) for rows in reader}

if len(pred_dict) != len(gt_dict):
    sys.exit("Test case length mismatch.")

episodic_acc = []
for key, value in pred_dict.items():
    if key not in gt_dict:
        sys.exit("Episodic id mismatch: \"{}\" does not exist in the provided ground truth file.".format(key))

    episodic_acc.append((gt_dict[key] == value).mean().item())

episodic_acc = np.array(episodic_acc)
mean = episodic_acc.mean()
std = episodic_acc.std()

print('Accuracy: {:.2f} +- {:.2f} %'.format(mean * 100, 1.96 * std / (600)**(1/2) * 100))

Accuracy: 43.99 +- 0.83 %


In [ ]:
import os
import sys
import argparse

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import Sampler

import csv
import random
import numpy as np
import pandas as pd
from argparse import Namespace

from PIL import Image
filenameToPILImage = lambda x: Image.open(x)

# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(SEED)
np.random.seed(SEED)

class Convnet(nn.Module):
    def __init__(self, in_channels=3, hid_channels=64, out_channels=64):
        super().__init__()
        self.encoder = nn.Sequential(
            conv_block(in_channels, hid_channels),
            conv_block(hid_channels, hid_channels),
            conv_block(hid_channels, hid_channels),
            conv_block(hid_channels, out_channels)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)

def conv_block(in_channels, out_channels):
    bn = nn.BatchNorm2d(out_channels)
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        bn,
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

class Protonet(nn.Module):
    def __init__(self, in_channels=3, hid_channels=64, out_channels=64):
        super().__init__()
        self.conv = Convnet(in_channels, hid_channels, out_channels)
        self.mlp = nn.Sequential(
            nn.Linear(1600, 800),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(800, 800),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(800, 400)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.mlp(x)
        return x

def worker_init_fn(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)

# mini-Imagenet dataset
class MiniDataset(Dataset):
    def __init__(self, csv_path, data_dir):
        self.data_dir = data_dir
        self.data_df = pd.read_csv(csv_path).set_index("id")

        self.transform = transforms.Compose([
            filenameToPILImage,
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

    def __getitem__(self, index):
        path = self.data_df.loc[index, "filename"]
        label = self.data_df.loc[index, "label"]
        image = self.transform(os.path.join(self.data_dir, path))
        return image, label

    def __len__(self):
        return len(self.data_df)

class GeneratorSampler(Sampler):
    def __init__(self, episode_file_path):
        episode_df = pd.read_csv(episode_file_path).set_index("episode_id")
        self.sampled_sequence = episode_df.values.flatten().tolist()

    def __iter__(self):
        return iter(self.sampled_sequence) 

    def __len__(self):
        return len(self.sampled_sequence)

def pairwise_distances(x, y, matching_fn='l2', parametric=None):
    n_x = x.shape[0]
    n_y = y.shape[0]

    if matching_fn == 'l2':
        distances = (
                x.unsqueeze(1).expand(n_x, n_y, -1) -
                y.unsqueeze(0).expand(n_x, n_y, -1)
        ).pow(2).sum(dim=2)
        return distances
    elif matching_fn == 'cosine':
        cos = nn.CosineSimilarity(dim=2, eps=1e-6)
        cosine_similarities = cos(x.unsqueeze(1).expand(n_x, n_y, -1), y.unsqueeze(0).expand(n_x, n_y, -1))

        return 1 - cosine_similarities
    elif matching_fn == 'parametric':
        x_exp = x.unsqueeze(1).expand(n_x, n_y, -1).reshape(n_x*n_y, -1)
        y_exp = y.unsqueeze(0).expand(n_x, n_y, -1).reshape(n_x*n_y, -1)
        
        distances = parametric(torch.cat([x_exp, y_exp], dim=-1))
        
        return distances.reshape(n_x, n_y)

    else:
        raise(ValueError('Unsupported similarity function'))

def predict(args, model, data_loader):
    for _, m in model.items():
        m.eval()

    prediction_results = []
    episodic_acc = []


    with torch.no_grad():
        # each batch represent one episode (support data + query data)
        for i, (data, target) in enumerate(data_loader):
            data = data.cuda()
            # split data into support and query data
            support_input = data[:args.N_way * args.N_shot,:,:,:] 
            query_input   = data[args.N_way * args.N_shot:,:,:,:]

            # create the relative label (0 ~ N_way-1) for query data
            # label_encoder = {target[i * args.N_shot] : i for i in range(args.N_way)}
            # query_label = torch.cuda.LongTensor([label_encoder[class_name] for class_name in target[args.N_way * args.N_shot:]])

            #  extract the feature of support and query data
            if args.type == 'proto':
                support = model['proto'](support_input)
                queries = model['proto'](query_input)

                #  calculate the prototype for each class according to its support data
                prototypes = support.reshape(args.N_way, args.N_shot, -1).mean(dim=1)

            if args.matching_fn == 'parametric':
                distances = pairwise_distances(queries, prototypes, args.matching_fn, model['parametric'])
            else:
                distances = pairwise_distances(queries, prototypes, args.matching_fn)

            else:
                support = model['cnn'](support_input)
                queries = model['cnn'](query_input)

                sample_idx = torch.tensor([torch.randint(args.N_shot * i, args.N_shot * (i + 1), (args.M_aug,)).numpy() for i in range(args.N_way)]).reshape(-1)
                sample = support[sample_idx]
                noise = torch.randn(sample.shape).cuda()

                support_g = model['g'](sample, noise).reshape(args.N_way, args.M_aug, -1)
                support = support.reshape(args.N_way, args.N_shot, -1)

                support_aug = torch.cat([support, support_g], dim=1)
                support_aug = support_aug.reshape(args.N_way * (args.N_shot + args.M_aug), -1)

                prototypes = model['mlp'](support_aug)
                prototypes = prototypes.reshape(args.N_way, args.N_shot + args.M_aug, -1).mean(dim=1)
                queries = model['mlp'](queries)

                distances = pairwise_distances(queries, prototypes, args.matching_fn)

            #  classify the query data depending on the its distense with each prototype
            y_pred = (-distances).softmax(dim=1).max(1, keepdim=True)[1]
            prediction_results.append(y_pred.reshape(-1))

    return prediction_results

def parse_args():
    # parser = argparse.ArgumentParser(description="Few shot learning")
    # parser.add_argument('--N-way', default=5, type=int, help='N_way (default: 5)')
    # parser.add_argument('--N-shot', default=1, type=int, help='N_shot (default: 1)')
    # parser.add_argument('--N-query', default=15, type=int, help='N_query (default: 15)')
    # parser.add_argument('--M_aug', default=10, type=int, help='M_augmentation (default: 10)')
    # parser.add_argument('--matching_fn', default='l2', type=str, help='distance matching function')
    # parser.add_argument('--load', type=str, help="Model checkpoint path")
    # parser.add_argument('--type', type=str, choices=['proto', 'dhm', 'improved'], help="Model type")
    # parser.add_argument('--test_csv', default='./hw4_data/val.csv', type=str, help="Testing images csv file")
    # parser.add_argument('--test_data_dir', default='./hw4_data/val', type=str, help="Testing images directory")
    # parser.add_argument('--testcase_csv', default='./hw4_data/val_testcase.csv', type=str, help="Test case csv")
    # parser.add_argument('--output_csv', type=str, help="Output filename")

    # return parser.parse_args()
    parameter = {
        'N_way': 5,
        'N_shot': 1,
        'N_query': 15,
        'M_aug': 10,
        'matching_fn': 'parametric',#cosine parametric l2
        'load': '/content/15600-protonet.pth',
        # 'type': 'proto',
        'test_csv': '/content/hw4_data/mini/val.csv',
        'test_data_dir': '/content/hw4_data/mini/val',
        'testcase_csv': '/content/hw4_data/mini/val_testcase.csv',
        'output_csv': './output6.csv',
    }
    config = Namespace(**parameter)
    return config

if __name__=='__main__':
    args = parse_args()

    test_dataset = MiniDataset(args.test_csv, args.test_data_dir)

    test_loader = DataLoader(
        test_dataset, batch_size=args.N_way * (args.N_query + args.N_shot),
        num_workers=3, pin_memory=False, worker_init_fn=worker_init_fn,
        sampler=GeneratorSampler(args.testcase_csv))

    # TODO: load your model
    state = torch.load(args.load)
    model = {}
    if args.type == 'proto':
        from prototypical_net.model import Protonet
        model['proto'] = Protonet().cuda()
        model['proto'].load_state_dict(state['state_dict'])

    if args.matching_fn == 'parametric':
        model['parametric'] = nn.Sequential(
            nn.Linear(800, 400),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(400, 1)
        ).cuda()
        model['parametric'].load_state_dict(state['parametric'])
    
    else:
        if args.type == 'dhm':
            from data_hallucination.model import Convnet, Hallucinator, MLP
        elif args.type == 'improved':
            from improved_data_hallucination.model import Convnet, Hallucinator, MLP

        model['cnn'] = Convnet().cuda()
        model['cnn'].load_state_dict(state['cnn'])
        model['g'] = Hallucinator().cuda()
        model['g'].load_state_dict(state['g'])
        model['mlp'] = MLP().cuda()
        model['mlp'].load_state_dict(state['mlp'])

    prediction_results = predict(args, model, test_loader)

    if os.path.dirname(args.output_csv):
        os.makedirs(os.path.dirname(args.output_csv), exist_ok=True)
    # TODO: output your prediction to csv
    with open(args.output_csv, 'w') as out_file:
        line = 'episode_id'
        for i in range(args.N_way*args.N_query):
            line += ',query%d' % (i)
        line += '\n'
        out_file.write(line)

        for i, prediction in enumerate(prediction_results):
            line = '%d' % (i)
            for j in prediction:
                line += ',%d' % j
            line += '\n'
            out_file.write(line)

In [ ]:
! cp -r /content/ckpt_cos/15600-protonet.pth /content/drive/MyDrive/DLCV/HW4/p1_models

Rreference: https://github.com/kai860115/DLCV2020-FALL/tree/main/hw4